#  Vasicek model calibration
**Ref** Chapter 7 of [Hir13]

Our goal is to design pricing engine and calirate vasicek model from data

## Pricing formula for Vasicek model

Notations: 
- $P(t, T)$: zero coupon bond price
- $L(t, T)$: LIBOR rate
- $s(t, T, N)$: swap rate with $N$ terms on $[t, T]$

Pricing formulas are 
$$L(t, T) = \frac{100}{T-t} ( \frac{1}{P(t, T)} - 1)$$
and 
$$ s(t, T, N) = 100 \frac{1 - P(t, T)}{ \Delta \sum_{j=1}^N P(t, t + j \Delta)}, \hbox{ where } \Delta = \frac{T - t}{N}. $$

If we further assume Vasicek model for the short rates with parameter $\theta = (\kappa, \mu, \sigma, r_0)$, i.e.
$$d r_t = \kappa (\mu - r_t) dt + dW_t, \hbox{ with } r_0,$$
then ZCB is determined by
$$P(t, T) = \mathbb E[e^{- \int_0^T r_t dt}] = e^{A(t, T) - B(t, T) r_t}$$
with 
$$B(t, T) = \frac{1 - e^{-\kappa(T-t)}}{\kappa}, \quad 
A(t, T) = (\mu - \frac{\sigma^2}{2\kappa^2}) [B(t, T) - (T-t)] - \frac{\sigma^2}{4\kappa} B^2(t, T).$$

__Todo 1__

Design pricing engine-1 of ZCB $P(0, T)$ using the above explicit formula.

__Todo 2__

Compute ZCB $P(0,1)$, Libor $L(0,1)$ with the following parameters:

In [16]:
theta = [.1, .05, .003, .03]
kappa, mu, sigma, r0 = theta

__Todo 3__
find 10 term swap rates with term length 1 year.

In [18]:
# function for swap rate s(0,T) with N terms and ZCB P(0, T_j)
# s(T, N, P): swap rate of s(0, T)
# T: tenor
# N: number of terms
# P: list of length N, with P[j-1] = P(0, T_j) for j = 1, 2, ... N
s = lambda T, N, P: 100 * (1 - P[N-1])/(T/N*np.sum(P))

In [19]:
N = 10
term = 1.
P = np.zeros(N)
for j in range(N):
    P[j] = ZCB(term*(j+1), theta)
    
print('Swap rate is ' + str(s(term*N, N, P)))

Swap rate is 3.7595005435913667


**ex.** Callibrate Vasicek model to the following data by SSRE.

In [20]:
dfLiborRate = pd.DataFrame({'maturity (months)': [1, 2, 3, 6, 12],
                            '20081029 rate(%)': [3.1175, 3.2738, 3.4200, 3.4275, 3.4213],
                            '20110214 rate(%)': [0.2647, 0.2890, 0.3140, 0.4657, 0.7975]
                          })

In [21]:
dfLiborRate

,20081029 rate(%),20110214 rate(%),maturity (months)
0,3.1175,0.2647,1
1,3.2738,0.2890,2
2,3.4200,0.3140,3
3,3.4275,0.4657,6
4,3.4213,0.7975,12


In [22]:
dfSwapRate = pd.DataFrame({'term (year)': [2, 3, 5, 7, 10, 15, 30],
                            '20081029 rate(%)': [2.6967, 3.1557, 3.8111, 4.1497, 4.3638, 4.3753, 4.2772],
                            '20110214 rate(%)': [1.0481, 1.5577, 2.5569, 3.1850, 3.7225, 4.1683, 4.4407]
                          })

In [23]:
dfSwapRate

,20081029 rate(%),20110214 rate(%),term (year)
0,2.6967,1.0481,2
1,3.1557,1.5577,3
2,3.8111,2.5569,5
3,4.1497,3.1850,7
4,4.3638,3.7225,10
5,4.3753,4.1683,15
6,4.2772,4.4407,30


In [24]:
#function for LiborRates with given parameter set thea and maturities (np.array) T_array in years
def LiborRate(T_array, theta):
    L = np.zeros(T_array.size)
    for i in range(T_array.size):
        T = T_array[i]
        P = ZCB(T, theta)
        L[i] = 100./T*(1./P -1)
    return L    
        
#Test    
npLibor = dfLiborRate.values
LiborRate(npLibor[:, 0]/12, theta)

array([3.03766908, 3.03955772, 3.04132432, 3.04141495, 3.04134003])

In [25]:
#function for generating swap rates
def SwapRate(term, TermNumArray, theta):
    max_N_term = int(np.max(TermNumArray)) 
    P = np.zeros(max_N_term)
    for i in range(max_N_term):
        P[i] = ZCB((i+1)*term, theta) 
    
    #initialize swap rates    
    SR = np.zeros(TermNumArray.size)
    for i in range(SR.size):
        N_term = int(npSwap[i,0])
        P_term = P[0:N_term]
        SR[i] = s(term*N_term, N_term, P_term)
    return SR


#Test         
npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array
SwapRate(term, TermNumArray, theta)

array([3.23662824, 3.32126082, 3.32126082, 3.39948061, 3.39948061,
       3.39948061, 3.39948061])

In [26]:
#function for SSRE
#x and y shall be np.array with the same length
def SSRE(x, y):
    return np.sum(np.power(x - y, 2)/np.power(y,2))

#test
x = np.arange(5)
y = np.arange(5,10)
SSRE(x,y)

2.9039155013857396

In [27]:
#For Oct 29, 2008 rate 

npLibor = dfLiborRate.values
T_array = npLibor[:, 0]/12


npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array

y1 = npLibor[:, 1]
y2 = npSwap[:,1]
y = np.append(y1, y2) #target

def iOut(theta):
    x1 = LiborRate(T_array, theta)
    x2 = SwapRate(term, TermNumArray, theta)
    x = np.append(x1,x2)
    return SSRE(x, y)
    
theta0 = [.1, .1, .00001, .01]    #initial start
min_theta = so.fmin(iOut, theta0, ftol = .0001, maxiter = 1000)
print('For Oct 29, 2008 rate is' + str(min_theta))
    

For Oct 29, 2008 rate is[ 6.92457301e-04  1.88692079e+01 -1.10453901e-03  1.40296645e-03]


In [28]:
#For Feb 14, 2011 rate 

npLibor = dfLiborRate.values
T_array = npLibor[:, 0]/12


npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array

y1 = npLibor[:, 2]
y2 = npSwap[:,2]
y = np.append(y1, y2) #target

def iOut(theta):
    x1 = LiborRate(T_array, theta)
    x2 = SwapRate(term, TermNumArray, theta)
    x = np.append(x1,x2)
    return SSRE(x, y)
    
theta0 = [.1, .1, .00001, .01]    #initial start
min_theta = so.fmin(iOut, theta0, ftol = .0001, maxiter = 1000)
print('For Feb 14, 2011 rate is' + str(min_theta))
    

For Feb 14, 2011 rate is[ 2.44600837e-04  7.63561265e+01 -2.55279651e-04  1.12926219e-02]


**Remark**
Above example is from Section 7.3.1.1 of [Hir13]. However, although the result from above calibration has three parameters matching the book, it is different for the volatility $\sigma$. I do not know why.